<a href="https://colab.research.google.com/github/whart222/pao/blob/master/doc/besancon27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A PAO TUTORIAL**

## Setup and Installation of PAO and GLPK

In [1]:
!pip install condacolab
import condacolab
condacolab.install()

  Using cached condacolab-0.1.2-py3-none-any.whl (6.0 kB)
✨🍰✨ Everything looks OK!


In [18]:
!conda install glpk ipopt

Solving environment: \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - glpk
    - ipopt


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ampl-mp-3.1.0              |    h616b090_1004         1.2 MB  conda-forge
    ipopt-3.13.4               |       h7ede334_0         1.1 MB  conda-forge
    libblas-3.9.0              |       8_openblas          11 KB  conda-forge
    libgfortran-ng-9.3.0       |      hff62375_18          22 KB  conda-forge
    libgfortran5-9.3.0         |      hff62375_18         2.0 MB  conda-forge
    liblapack-3.9.0            |       8_openblas          11 KB  conda-forge
    libopenblas-0.3.12         |pthreads_h4812303_1         8.9 MB  conda-forge
    metis-5.1.0                |    h58526e2_1006         4.1 MB  conda-forge
    mumps-include-5.2.1      

In [3]:
!git clone https://github.com/or-fusion/pao.git
!pip install -e /content/pao

fatal: destination path 'pao' already exists and is not an empty directory.
Obtaining file:///content/pao
  Attempting uninstall: pao
    Found existing installation: pao 1.0.dev0
    Uninstalling pao-1.0.dev0:
      Successfully uninstalled pao-1.0.dev0
  Running setup.py develop for pao


## Creating and Solving a PAO Model using Pyomo

In [4]:
import pyomo.environ as pe
from pao.bilevel import *

In [21]:
# Example 2.7 from
#
# Near-Optimal Robust Bilevel Optimization
#   M. Besancon, M. F. Anjos and L. Brotcorne
#   arXiv:1908.04040v5 (2019)
#
# Optimal solution: (x,v) = (0,1)

M = pe.ConcreteModel()

M.x = pe.Var(bounds=(0.5,None),initialize=1)
M.v = pe.Var(initialize=1)

M.o = pe.Objective(expr=M.x, sense=pe.minimize)
M.c = pe.Constraint(expr= M.v >= 1 - M.x/10)

M.L = SubModel(fixed=M.x)
M.L.o = pe.Objective(expr=M.v, sense=pe.maximize)
M.L.c = pe.Constraint(expr= 1 + M.x/10 >= M.v)

In [6]:
# Available Solvers
print(list(SolverFactory))

['pao.lbp.FA', 'pao.lbp.PCCG', 'pao.lbp.REG', 'pao.lbp.interdiction', 'pao.pyomo.FA', 'pao.pyomo.PCCG', 'pao.pyomo.REG']


In [25]:
solver = SolverFactory("pao.pyomo.FA")
results = solver.solve(M, solver="glpk")

In [10]:
# The results object contains information about the problem and solver
# print(results)

In [26]:
# The solution is loaded back into the model
M.pprint()

1 Set Declarations
    zzz_PAO_SlackVariables_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    0 :      {}

3 Var Declarations
    v : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :  None : 0.94999999999709 :  None : False : False :  Reals
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :   0.5 :   0.0 :  None : False : False :  Reals
    zzz_PAO_SlackVariables : Size=0, Index=zzz_PAO_SlackVariables_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain

1 Objective Declarations
    o : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize :          x

1 Constraint Declarations
    c : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  -Inf : 1 - 0.1*x - v :   0.0 :   True

1 S